In [1]:
import os
from configparser import ConfigParser
path_config_file = os.getcwd().split('docrv2_sroie')[0] + 'docrv2_sroie/' + 'config.ini'
common_variable = ConfigParser()
common_variable.read(path_config_file)

print(common_variable.sections())
print(common_variable['database']['module_path'])
print(common_variable['database']['DB_BASE'])

import sys
sys.path.append(common_variable['database']['module_path'])
from db_util import *
from db import *

import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

from parse import parse
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from glob import glob
from tqdm import tqdm_notebook

['database', 'mlflow', 'font']
/media/myungsungkwak/msdisk/docrv2_sroie/modules/database/
/media/myungsungkwak/msdisk/docrv2_data/DB/


In [2]:
def create_database(db_idx, db_name=None):
    path = common_variable['database']['DB_BASE']
    os.makedirs(path, exist_ok=True)

    name = db_name + "_%04d"%db_idx
    description = 'data db from train_task2 all coords images'
    datadb = create_data_db(path=path, name=name, description=description, data_class='DATA')
    print("... created db : ", name)
    return datadb

def update_database(datadb, imgs, coords, refname, db_start_idx):
    for idx, img in enumerate(imgs):
        chars = []
        chars.append(coords[idx][8])
        char_table = pd.DataFrame(coords[idx])
        char_table = char_table.T
        char_table.rename(columns={0:'x1', 1:'y1', 2:'x2', 3:'y2', 4:'x3',
                                   5:'y3', 6:'x4', 7:'y4', 8:'char'}, inplace=True)
        
        char_table.x1 = char_table.x1.astype(int)
        char_table.y1 = char_table.y1.astype(int)
        char_table.x2 = char_table.x2.astype(int)
        char_table.y2 = char_table.y2.astype(int)
        char_table.x3 = char_table.x3.astype(int)
        char_table.y3 = char_table.y3.astype(int)
        char_table.x4 = char_table.x4.astype(int)
        char_table.y4 = char_table.y4.astype(int)
        
        update_data(datadb, index=db_start_idx+idx, label='task12', image=img, text=chars[0], ref=refname)

def get_coords_from_image(txt_file):
    lps = []

    try:
        with open(txt_file, 'r') as f:
            coords_data = f.read().split('\n')
            for ii in range(len(coords_data)):
                if len(coords_data[ii]) > 0:
                    ps = parse('{},{},{},{},{},{},{},{},{}', coords_data[ii])
                    ps = list(ps)
                    ps[:-1] = list(map(int, ps[:-1]))
                    lps.append(ps)
    except:
        with open(txt_file, 'r', encoding='ISO-8859-1') as f:
            coords_data = f.read().split('\n')
            for ii in range(len(coords_data)):
                if len(coords_data[ii]) > 0:
                    ps = parse('{},{},{},{},{},{},{},{},{}', coords_data[ii])
                    ps = list(ps)
                    ps[:-1] = list(map(int, ps[:-1]))
                    lps.append(ps)
    return lps

In [3]:
"""
method 1 - no padding
crop images followed as original coordiantes
"""
def get_crop_image_by_coords(img_path, coords, num_images=1):
    img = Image.open(img_path)
    o_width = img.size[0]
    o_height = img.size[1]
    
    transpose_x1_coords = np.transpose(coords)[0]
    transpose_y1_coords = np.transpose(coords)[1]
    transpose_x1_coords = list(map(int, transpose_x1_coords)) 
    transpose_y1_coords = list(map(int, transpose_y1_coords))
    
    transpose_x2_coords = np.transpose(coords)[2]
    transpose_y2_coords = np.transpose(coords)[3]
    transpose_x2_coords = list(map(int, transpose_x2_coords)) 
    transpose_y2_coords = list(map(int, transpose_y2_coords))    
    
    transpose_x3_coords = np.transpose(coords)[4]
    transpose_y3_coords = np.transpose(coords)[5]
    transpose_x3_coords = list(map(int, transpose_x3_coords)) 
    transpose_y3_coords = list(map(int, transpose_y3_coords))

    transpose_x4_coords = np.transpose(coords)[6]
    transpose_y4_coords = np.transpose(coords)[7]
    transpose_x4_coords = list(map(int, transpose_x4_coords)) 
    transpose_y4_coords = list(map(int, transpose_y4_coords))
    
    transpose_chars = np.transpose(coords)[8]
    
    sample_list = []
    coords_list = []
    for idx in range(num_images):
        base_idx = idx

        new_x1 = transpose_x1_coords[base_idx]
        new_y1 = transpose_y1_coords[base_idx]

        new_x2 = transpose_x2_coords[base_idx]
        new_y2 = transpose_y2_coords[base_idx]

        new_x3 = transpose_x3_coords[base_idx]
        new_y3 = transpose_y3_coords[base_idx]

        new_x4 = transpose_x4_coords[base_idx]
        new_y4 = transpose_y4_coords[base_idx]
    
        crop_img = img.crop((new_x1,
                             new_y1,
                             new_x3,
                             new_y3))
        sample_list.append(crop_img)
        coords_list.append([new_x1, new_y1, new_x2, new_y2, new_x3, new_y3, new_x4, new_y4,
                           transpose_chars[base_idx]])

    return sample_list, coords_list

In [4]:
"""
method 2 - random size padding
crop images followed as original coordiantes
"""
def get_crop_image_by_coords_pad_random(img_path, coords, num_images=1):
    img = Image.open(img_path)
    o_width = img.size[0]
    o_height = img.size[1]

    transpose_x1_coords = np.transpose(coords)[0]
    transpose_y1_coords = np.transpose(coords)[1]
    transpose_x1_coords = list(map(int, transpose_x1_coords))
    transpose_y1_coords = list(map(int, transpose_y1_coords))

    transpose_x2_coords = np.transpose(coords)[2]
    transpose_y2_coords = np.transpose(coords)[3]
    transpose_x2_coords = list(map(int, transpose_x2_coords))
    transpose_y2_coords = list(map(int, transpose_y2_coords))

    transpose_x3_coords = np.transpose(coords)[4]
    transpose_y3_coords = np.transpose(coords)[5]
    transpose_x3_coords = list(map(int, transpose_x3_coords))
    transpose_y3_coords = list(map(int, transpose_y3_coords))

    transpose_x4_coords = np.transpose(coords)[6]
    transpose_y4_coords = np.transpose(coords)[7]
    transpose_x4_coords = list(map(int, transpose_x4_coords))
    transpose_y4_coords = list(map(int, transpose_y4_coords))

    transpose_chars = np.transpose(coords)[8]
    
    sample_list = []
    coords_list = []
    
    img_cnt = 0

    for _ in range(np.random.randint(1, 10)):
        # [-4  5  9  3] --> left right up bottom
        
        # num_images --> coords imags
        for idx in range(num_images):
            pdd = np.random.randint(-5,10,4)
            base_idx = idx
            while True:
                if (transpose_x3_coords[base_idx] - transpose_x1_coords[base_idx] <= 20) or (transpose_y3_coords[base_idx] - transpose_x1_coords[base_idx]<=20):
                    new_x1 = transpose_x1_coords[base_idx]
                    new_y1 = transpose_y1_coords[base_idx]
                    new_x2 = transpose_x2_coords[base_idx]
                    new_y2 = transpose_y2_coords[base_idx]
                    new_x3 = transpose_x3_coords[base_idx]
                    new_y3 = transpose_y3_coords[base_idx]
                    new_x4 = transpose_x4_coords[base_idx]
                    new_y4 = transpose_y4_coords[base_idx]
                    break
                else:
                    new_x1 = transpose_x1_coords[base_idx] - pdd[0] # pad left
                    new_y1 = transpose_y1_coords[base_idx] - pdd[2] # pad up

                    new_x3 = transpose_x3_coords[base_idx] + pdd[1] # pad right
                    new_y3 = transpose_y3_coords[base_idx] + pdd[3] # pad bottom

                    new_x2 = new_x3
                    new_y2 = new_y1

                    new_x4 = new_x1
                    new_y4 = new_y3

                    if (new_x1 < 0) or (new_x4 < 0) or (new_x2 > o_width) or (new_x3 > o_width):
                        print("[problem] x coords")
                        pdd = np.random.randint(-5,10,4)
                        continue
                    if (new_y1 < 0) or (new_y2 < 0) or (new_y3 > o_height) or (new_y3 > o_height):
                        print("[problem] y coords")
                        pdd = np.random.randint(-5,10,4)
                        continue
                    break

            crop_img = img.crop((new_x1, new_y1, new_x3, new_y3))
            if (crop_img.size[0]) > 0 and (crop_img.size[1] > 0):
                sample_list.append(crop_img)
                coords_list.append([new_x1, new_y1, new_x2, new_y2, new_x3, new_y3, new_x4, new_y4,
                                    transpose_chars[base_idx]])

                img_cnt = img_cnt + 1

    return sample_list, coords_list, img_cnt

In [6]:
"""
method 3 - fixed size padding
crop images followed as original coordiantes
"""
def get_crop_image_by_coords_pad(img_path, coords, num_images=1, pad_max=5):
    img = Image.open(img_path)
    o_width = img.size[0]
    o_height = img.size[1]

    transpose_x1_coords = np.transpose(coords)[0]
    transpose_y1_coords = np.transpose(coords)[1]
    transpose_x1_coords = list(map(int, transpose_x1_coords))
    transpose_y1_coords = list(map(int, transpose_y1_coords))

    transpose_x2_coords = np.transpose(coords)[2]
    transpose_y2_coords = np.transpose(coords)[3]
    transpose_x2_coords = list(map(int, transpose_x2_coords))
    transpose_y2_coords = list(map(int, transpose_y2_coords))

    transpose_x3_coords = np.transpose(coords)[4]
    transpose_y3_coords = np.transpose(coords)[5]
    transpose_x3_coords = list(map(int, transpose_x3_coords))
    transpose_y3_coords = list(map(int, transpose_y3_coords))

    transpose_x4_coords = np.transpose(coords)[6]
    transpose_y4_coords = np.transpose(coords)[7]
    transpose_x4_coords = list(map(int, transpose_x4_coords))
    transpose_y4_coords = list(map(int, transpose_y4_coords))

    transpose_chars = np.transpose(coords)[8]
    
    """
    pad_type
    1) left ←
    2) right →
    3) up ↑
    4) bottom ↓
    5) all ←, →, ↑, ↓
    """
    # 1~5
    # p_size = np.random.randint(1, pad_max)
    p_size = pad_max # only 5 pix
    #pad_type = ["left", "right", "up", "bottom", "all"]
    pad_type = ["all"]

    sample_list = []
    coords_list = []
    
    for pad_idx in range(len(pad_type)):
        for idx in range(num_images):
            base_idx = idx

            new_x1 = transpose_x1_coords[base_idx]
            new_y1 = transpose_y1_coords[base_idx]

            new_x2 = transpose_x2_coords[base_idx]
            new_y2 = transpose_y2_coords[base_idx]

            new_x3 = transpose_x3_coords[base_idx]
            new_y3 = transpose_y3_coords[base_idx]

            new_x4 = transpose_x4_coords[base_idx]
            new_y4 = transpose_y4_coords[base_idx]
            
            if pad_type[pad_idx] == "left":
                new_x1 = new_x1 - p_size
                new_x4 = new_x4 - p_size
            elif pad_type[pad_idx] == "right":
                new_x2 = new_x2 + p_size
                new_x3 = new_x3 + p_size
            elif pad_type[pad_idx] == "up":
                new_y1 = new_y1 - p_size
                new_y2 = new_y2 - p_size
            elif pad_type[pad_idx] == "bottom":
                new_y3 = new_y3 + p_size
                new_y4 = new_y4 + p_size
            else:
                new_x1 = new_x1 - p_size
                new_y1 = new_y1 - p_size
                
                new_x2 = new_x2 + p_size
                new_y2 = new_y2 - p_size
                
                new_x3 = new_x3 + p_size
                new_y3 = new_y3 + p_size
                
                new_x4 = new_x4 - p_size
                new_y4 = new_y4 + p_size
                
                
            crop_img = img.crop((new_x1, new_y1, new_x3, new_y3))
            sample_list.append(crop_img)
            coords_list.append([new_x1, new_y1, new_x2, new_y2, new_x3, new_y3, new_x4, new_y4,
                                transpose_chars[base_idx]])

    return sample_list, coords_list

In [10]:
"""
temp
"""
def get_crop_image_by_coords_2(img_path, coords, num_images=1):
    img = Image.open(img_path)
    o_width = img.size[0]
    o_height = img.size[1]
    
    transpose_x1_coords = np.transpose(coords)[0]
    transpose_y1_coords = np.transpose(coords)[1]
    transpose_x1_coords = list(map(int, transpose_x1_coords)) 
    transpose_y1_coords = list(map(int, transpose_y1_coords))
    
    transpose_x2_coords = np.transpose(coords)[2]
    transpose_y2_coords = np.transpose(coords)[3]
    transpose_x2_coords = list(map(int, transpose_x2_coords)) 
    transpose_y2_coords = list(map(int, transpose_y2_coords))    
    
    transpose_x3_coords = np.transpose(coords)[4]
    transpose_y3_coords = np.transpose(coords)[5]
    transpose_x3_coords = list(map(int, transpose_x3_coords)) 
    transpose_y3_coords = list(map(int, transpose_y3_coords))

    transpose_x4_coords = np.transpose(coords)[6]
    transpose_y4_coords = np.transpose(coords)[7]
    transpose_x4_coords = list(map(int, transpose_x4_coords)) 
    transpose_y4_coords = list(map(int, transpose_y4_coords))
    
    transpose_chars = np.transpose(coords)[8]
    
    pad_size = 5
    sample_list = []
    coords_list = []
    for idx in range(num_images):
        base_idx = idx

        new_x1 = transpose_x1_coords[base_idx] - pad_size
        new_y1 = transpose_y1_coords[base_idx] - pad_size

        new_x2 = transpose_x2_coords[base_idx] + pad_size
        new_y2 = transpose_y2_coords[base_idx] - pad_size

        new_x3 = transpose_x3_coords[base_idx] + pad_size
        new_y3 = transpose_y3_coords[base_idx] + pad_size

        new_x4 = transpose_x4_coords[base_idx] - pad_size
        new_y4 = transpose_y4_coords[base_idx] + pad_size
    
        crop_img = img.crop((new_x1,
                             new_y1,
                             new_x3,
                             new_y3))
        sample_list.append(crop_img)
        coords_list.append([new_x1, new_y1, new_x2, new_y2, new_x3, new_y3, new_x4, new_y4,
                           transpose_chars[base_idx]])

    return sample_list, coords_list

"""
run1: create data - no padding (use get_crop_image_by_coords)
"""
datadir = '/media/myungsungkwak/msdisk/docrv2_sroie/data/test_task12_img_text/'
img_files = glob(datadir+'*.jpg')
txt_files = glob(datadir+'*.txt')

crop_images = []
coords_table = []

# create db : db_name_0001~000N
datadb = create_database(1, "recog_test_all_pad5_valid")
num_images = 1
db_start_idx = 0
for idx in tqdm(range(len(img_files))):
    refname = os.path.basename(img_files[idx])[:-4]
    txt_filepath = os.path.join(datadir, refname+'.txt')
    coords_table.append(get_coords_from_image(txt_filepath))

    num_images = len(coords_table[idx])

    crop_img, crop_coords = get_crop_image_by_coords_2(img_files[idx], coords_table[idx], num_images=num_images)
    update_database(datadb, crop_img, crop_coords, refname, db_start_idx)
    db_start_idx = db_start_idx + num_images

del coords_table[:][:]

  0%|          | 0/361 [00:00<?, ?it/s]

... created db :  recog_test_all_pad5_valid_0001


100%|██████████| 361/361 [40:18<00:00,  6.70s/it]


In [20]:
"""
test
"""
valid_db_name = "recog_test_task2_all_0001"
valid_db = open_env(os.path.join(common_variable['database']['DB_BASE'],valid_db_name))
imgs = read_bulk_data_from_db(valid_db, prefix='img')

In [21]:
len(imgs)

5989

In [ ]:
"""
run2: create data - random size padding (use get_crop_image_by_coords_pad_random)
"""
for folder_idx in range(4):
    datadir = '/media/myungsungkwak/msdisk/docrv2_sroie/data/train_task2_crop_split_{}/'.format(folder_idx+1)
    print("[" + datadir + "]")
    img_files = glob(datadir+'*.jpg')
    txt_files = glob(datadir+'*.txt')

    crop_images = []
    coords_table = []

    datadb = create_database(folder_idx+1, "recog_train_task2_random_pad_sp")
    num_images = 1
    db_start_idx = 0
    for idx in range(len(img_files)):
        
        refname = os.path.basename(img_files[idx])[:-4]
        txt_filepath = os.path.join(datadir, refname+'.txt')
        coords_table.append(get_coords_from_image(txt_filepath))
        
        print("{}. {}".format(idx, img_files[idx]))
        print("{}. {}".format(idx, txt_filepath))

        num_images = len(coords_table[idx])

        # crop_img, crop_coords = get_crop_image_by_coords(img_files[idx], coords_table[idx], num_images=num_images)
        crop_img, crop_coords, img_cnt = get_crop_image_by_coords_pad_random(img_files[idx], 
                                                                             coords_table[idx], 
                                                                             num_images=num_images)
        
        print("- orig imgs : {}, len(crimg) : {}, created imgs : {}, start_db_idx : {}, end_db_idx : {}".format(
            num_images,
            len(crop_img),
            img_cnt,
            db_start_idx,
            db_start_idx+img_cnt))
        
        update_database(datadb, crop_img, crop_coords, refname, db_start_idx)
        
        db_start_idx = db_start_idx + img_cnt
        

    del coords_table[:][:]
print("...completed")

In [7]:
"""
run3: create data - fixed size padding (use get_crop_image_by_coords_pad)
left, right, up, bottom, all
"""
for folder_idx in range(1):
    #datadir = '/media/myungsungkwak/msdisk/docrv2_sroie/data/train_task2_crop_split_{}/'.format(folder_idx+1)
    datadir = '/media/myungsungkwak/msdisk/docrv2_sroie/data/test_task12_img_text/'
    print("[" + datadir + "]")
    img_files = glob(datadir+'*.jpg')
    txt_files = glob(datadir+'*.txt')

    crop_images = []
    coords_table = []

    datadb = create_database(folder_idx+1, "recog_test_task2_all_pad5")
    num_images = 1
    db_start_idx = 0
    for idx in tqdm(range(len(img_files))):
        # print(idx, " : ", img_files[idx])
        refname = os.path.basename(img_files[idx])[:-4]
        txt_filepath = os.path.join(datadir, refname+'.txt')
        coords_table.append(get_coords_from_image(txt_filepath))

        num_images = len(coords_table[idx])

        crop_img, crop_coords = get_crop_image_by_coords_pad(img_files[idx], coords_table[idx], 
                                                             num_images=num_images)
        update_database(datadb, crop_img, crop_coords, refname, db_start_idx)
        db_start_idx = db_start_idx + (num_images * 5)

    del coords_table[:][:]

  0%|          | 0/361 [00:00<?, ?it/s]

[/media/myungsungkwak/msdisk/docrv2_sroie/data/test_task12_img_text/]
... created db :  recog_test_task2_all_pad5_0001


100%|██████████| 361/361 [42:08<00:00,  7.00s/it]


In [8]:
!ls "/media/myungsungkwak/msdisk/docrv2_data/DB/"

d2gan_train_task12_crop50080_0002
d2gan_train_task12_crop_0000
d2gan_train_task12_crop_0001
d2gan_train_task12_crop_random_0001
d2gan_train_task12_crop_random_0002
db_main
inference_crop_0831_thr05_all
inference_crop_0831_thr05_all_preview_detection
inference_crop_0831_thr05_sample40
inference_crop_0831_thr05_sample40_preview_detection
inference_crop_0831_thr07_sample40
inference_crop_0831_thr07_sample40_preview_detection
inference_crop_0901_thr05_sample40
inference_crop_0901_thr05_sample40_preview_detection
raw_recog_inference_data_all_0001
raw_test_task12_img_txt
raw_test_task12_img_txt_0001
raw_test_task12_img_txt_X51005230605
raw_test_task12_img_txt_issue
raw_test_task12_img_txt_sample
raw_test_task12_img_txt_sample40
recog_inference_result_all_0001
recog_test_task2_all_0001
recog_test_task2_all_pad5_0001
recog_test_task2_eval_0001
recog_train_task2_all_0001
recog_train_task2_pad_sp_0001
recog_train_task2_pad_sp_0002
recog_train_task2_pad_sp_0003
recog_train_task2_pad_sp_0004
recog